In [1]:
# in order to install the required packages for the snn conversion, 
# you need the following packages
# furthemore you must have installed miniconda or anaconda
# and activated a virtual environment to execute the following commands

# %conda install akida
# %conda install cnn2snn
# %conda install akida-models

In [2]:
import tensorflow as tf
import utils

h:\Programme\Miniconda\envs\studienarbeit\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds_train, ds_train_info = utils.getDataset('train')
ds_test, ds_test_info = utils.getDataset('test')
ds_eval, ds_eval_info = utils.getDataset('validation')





 Tensor("args_0:0", shape=(None, 4), dtype=float32) 

 Tensor("strided_slice:0", shape=(), dtype=int32) 






 Tensor("args_0:0", shape=(None, 4), dtype=float32) 

 Tensor("strided_slice:0", shape=(), dtype=int32) 






 Tensor("args_0:0", shape=(None, 4), dtype=float32) 

 Tensor("strided_slice:0", shape=(), dtype=int32) 




In [4]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_file
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential

In [5]:
#model_file = get_file(
#    "akidanet_imagenet_224_alpha_50.h5",
#    "http://data.brainchip.com/models/akidanet/akidanet_imagenet_224_alpha_50.h5",
#    cache_subdir='models/akidanet_imagenet')
#
#model = Sequential()
#model.add(load_model(model_file)) 
#model.add(Dense(4, activation='relu'))

In [6]:
#model.compile(optimizer='adam',
#              loss=tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"), 
#              metrics=['accuracy'])
#model.get_layer(index=-1)
#
#
#cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./imagenet_models",
#                                                 save_weights_only=True,
#                                                 verbose=1)

In [7]:
#model.fit(
#    ds_train,
#    epochs=6,
#    validation_data=ds_test,
#)

In [8]:
#results = model.evaluate(
#  ds_eval
#)
#
#print("test loss, test acc:", results)
#
#model.save("models/imagenet_model")

In [9]:
import cv2
import numpy as np

model = tf.keras.models.load_model("models/imagenet_model")

In [17]:
bbox = None
image = None

for ele in ds_eval.take(1):
  
  image = np.array(ele[0])
  bbox = np.array(ele[1])
  
  cv2.imshow("bounding_box", image[0])
  print(image[0].shape)
  print(bbox)
  break

(224, 224, 3)
[[[0.05405406 0.6855469  0.30743244 0.7792969 ]]

 [[0.2693997  0.6269531  0.4216691  0.7109375 ]]

 [[0.015625   0.4296875  0.45052084 0.6640625 ]]

 [[0.05       0.01171875 0.46470588 0.23046875]]

 [[0.13765182 0.37109375 0.7611336  0.69921875]]

 [[0.06428572 0.36523438 0.42857143 0.546875  ]]

 [[0.26548672 0.4296875  0.5958702  0.59375   ]]

 [[0.14035088 0.20703125 0.35672516 0.3046875 ]]

 [[0.15075377 0.31933594 0.50628144 0.8173828 ]]

 [[0.29868227 0.49804688 0.66178626 0.6816406 ]]

 [[0.19590643 0.3203125  0.86549705 0.65625   ]]

 [[0.16666667 0.42382812 0.44444445 0.5996094 ]]

 [[0.2326389  0.46972656 0.2986111  0.5       ]]

 [[0.16983895 0.58203125 0.3338214  0.6582031 ]]

 [[0.13802083 0.7363281  0.29427084 0.83984375]]

 [[0.15040651 0.5390625  0.63821137 0.71484375]]

 [[0.07426161 0.31152344 0.292827   0.48046875]]

 [[0.13065693 0.32910156 0.47591242 0.6738281 ]]

 [[0.03712036 0.53222656 0.22665916 0.7626953 ]]

 [[0.13896103 0.35253906 0.42857143 

: 

In [10]:
prediction = model.predict(
  image
)

NameError: name 'image' is not defined

In [ ]:

for box in [bbox, prediction]:

  x = box[0]
  y = box[1]
  xx = box[2]
  yy = box[3]

  print("x ", x, "y ", y, "xx ", xx, "yy", yy)

  cv2.rectangle(image, (x, y), (xx, yy), (0, 0, 255), 2)
  print("x,y,w,h:",x,y,xx,yy)
  
# save resulting image
cv2.imwrite('example.jpg', image)      

# show thresh and result    
cv2.imshow("bounding_box", image)
cv2.waitKey(0)
cv2.destroyAllWindows()